# 1. Carga & perfilado rápido

In [60]:
import pandas as pd
import numpy as np

df = pd.read_csv("/Users/andres/Desktop/PDD/ncr_ride_bookings.csv")

print(df.shape)
print(df.info())
print(df.head(5))
print(df.describe(include="all"))

(150000, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 21 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Date                               150000 non-null  object 
 1   Time                               150000 non-null  object 
 2   Booking ID                         150000 non-null  object 
 3   Booking Status                     150000 non-null  object 
 4   Customer ID                        150000 non-null  object 
 5   Vehicle Type                       150000 non-null  object 
 6   Pickup Location                    150000 non-null  object 
 7   Drop Location                      150000 non-null  object 
 8   Avg VTAT                           139500 non-null  float64
 9   Avg CTAT                           102000 non-null  float64
 10  Cancelled Rides by Customer        10500 non-null   float64
 11  Reason for cancelling by C

# 2. Estandarización

In [61]:
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
df['date'] = pd.to_datetime(df['date'])
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S').dt.time

numeric_columns = ['avg_vtat', 'avg_ctat', 'cancelled_rides_by_customer', 
                   'cancelled_rides_by_driver', 'incomplete_rides', 
                   'booking_value', 'ride_distance', 'driver_ratings', 'customer_rating']

categorical_columns = ['booking_status', 'vehicle_type', 'pickup_location', 'drop_location',
                       'reason_for_cancelling_by_customer', 'driver_cancellation_reason',
                       'incomplete_rides_reason', 'payment_method']

for col in categorical_columns:
    df[col] = df[col].astype(str).str.lower().str.strip().str.replace(' ', '_')

id_columns = ['booking_id', 'customer_id']
for col in id_columns:
    df[col] = df[col].astype(str).str.strip().str.replace('"', '')

In [62]:
print(df.shape)
print(df.info())
print(df.head(5))
print(df.describe(include="all"))

(150000, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 21 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   date                               150000 non-null  datetime64[ns]
 1   time                               150000 non-null  object        
 2   booking_id                         150000 non-null  object        
 3   booking_status                     150000 non-null  object        
 4   customer_id                        150000 non-null  object        
 5   vehicle_type                       150000 non-null  object        
 6   pickup_location                    150000 non-null  object        
 7   drop_location                      150000 non-null  object        
 8   avg_vtat                           139500 non-null  float64       
 9   avg_ctat                           102000 non-null  float64       
 10  cancell

# 3. Tiempo & Features

In [63]:
df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str))

df['hour'] = df['datetime'].dt.hour
df['day_of_week'] = df['datetime'].dt.day_name()
df['month'] = df['datetime'].dt.month
df['quarter'] = df['datetime'].dt.quarter

duplicates = df.duplicated(subset=['booking_id'], keep=False)
df = df.drop_duplicates(subset=['booking_id'], keep='first')
remaining_duplicates = df.duplicated(subset=['booking_id']).sum()

# 4. Limpieza

In [64]:
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

numeric_cols = ['avg_vtat', 'avg_ctat', 'booking_value', 'ride_distance', 'driver_ratings', 'customer_rating']

print("Análisis de outliers por columna:")
for col in numeric_cols:
    if df[col].notna().sum() > 0:
        outliers, lower, upper = detect_outliers_iqr(df, col)
        print(f"\n{col}:")
        print(f"  - Límite inferior: {lower:.2f}")
        print(f"  - Límite superior: {upper:.2f}")
        print(f"  - Outliers encontrados: {len(outliers)}")
        print(f"  - Porcentaje de outliers: {(len(outliers)/len(df)*100):.2f}%")


Análisis de outliers por columna:

avg_vtat:
  - Límite inferior: -3.70
  - Límite superior: 20.30
  - Outliers encontrados: 0
  - Porcentaje de outliers: 0.00%

avg_ctat:
  - Límite inferior: -1.20
  - Límite superior: 59.60
  - Outliers encontrados: 0
  - Porcentaje de outliers: 0.00%

booking_value:
  - Límite inferior: -448.50
  - Límite superior: 1371.50
  - Outliers encontrados: 3410
  - Porcentaje de outliers: 2.29%

ride_distance:
  - Límite inferior: -24.08
  - Límite superior: 73.36
  - Outliers encontrados: 0
  - Porcentaje de outliers: 0.00%

driver_ratings:
  - Límite inferior: 3.35
  - Límite superior: 5.35
  - Outliers encontrados: 5162
  - Porcentaje de outliers: 3.47%

customer_rating:
  - Límite inferior: 3.30
  - Límite superior: 5.70
  - Outliers encontrados: 3237
  - Porcentaje de outliers: 2.18%


In [65]:
from scipy import stats

def detect_outliers_zscore(df, column, threshold=3):
    z_scores = np.abs(stats.zscore(df[column].dropna()))
    outliers = df[column].dropna()[z_scores > threshold]
    return outliers

print("\n\nDetección de outliers usando Z-score (threshold=3):")
for col in numeric_cols:
    if df[col].notna().sum() > 0:
        outliers_z = detect_outliers_zscore(df, col)
        print(f"{col}: {len(outliers_z)} outliers extremos")



Detección de outliers usando Z-score (threshold=3):
avg_vtat: 196 outliers extremos
avg_ctat: 0 outliers extremos
booking_value: 1429 outliers extremos
ride_distance: 0 outliers extremos
driver_ratings: 0 outliers extremos
customer_rating: 467 outliers extremos


In [66]:
df_clean = df.copy()

print(f"Registros antes de eliminar outliers: {len(df_clean)}")

def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    mask = (df[column] >= lower_bound) & (df[column] <= upper_bound)
    return df[mask], lower_bound, upper_bound

numeric_cols = ['avg_vtat', 'avg_ctat', 'booking_value', 'ride_distance', 'driver_ratings', 'customer_rating']

Registros antes de eliminar outliers: 148767


In [67]:
for col in numeric_cols:
    if df_clean[col].notna().sum() > 0:
        initial_count = len(df_clean)
        df_clean, lower, upper = remove_outliers_iqr(df_clean, col)
        removed_count = initial_count - len(df_clean)
        print(f"{col}:")
        print(f"  - Límites: [{lower:.2f}, {upper:.2f}]")
        print(f"  - Registros eliminados: {removed_count}")
        print(f"  - Registros restantes: {len(df_clean)}")

print(f"\nRegistros después de eliminar outliers: {len(df_clean)}")
print(f"Total de registros eliminados: {len(df) - len(df_clean)}")
print(f"Porcentaje de datos conservados: {(len(df_clean)/len(df)*100):.2f}%")

# Comparar estadísticas antes y después
print("\nComparación de estadísticas:")
print("ANTES (con outliers):")
print(df[numeric_cols].describe())
print("\nDESPUÉS (sin outliers):")
print(df_clean[numeric_cols].describe())

avg_vtat:
  - Límites: [-3.70, 20.30]
  - Registros eliminados: 10401
  - Registros restantes: 138366
avg_ctat:
  - Límites: [-1.20, 59.60]
  - Registros eliminados: 37191
  - Registros restantes: 101175
booking_value:
  - Límites: [-448.50, 1371.50]
  - Registros eliminados: 3410
  - Registros restantes: 97765
ride_distance:
  - Límites: [-24.09, 73.35]
  - Registros eliminados: 0
  - Registros restantes: 97765
driver_ratings:
  - Límites: [3.35, 5.35]
  - Registros eliminados: 13623
  - Registros restantes: 84142
customer_rating:
  - Límites: [3.30, 5.70]
  - Registros eliminados: 2969
  - Registros restantes: 81173

Registros después de eliminar outliers: 81173
Total de registros eliminados: 67594
Porcentaje de datos conservados: 54.56%

Comparación de estadísticas:
ANTES (con outliers):
            avg_vtat       avg_ctat  booking_value  ride_distance  \
count  138366.000000  101175.000000  101175.000000  101175.000000   
mean        8.454819      29.150249     508.290230      24.6

In [68]:
print(f"Registros después de eliminar outliers: {len(df_clean)}")

Registros después de eliminar outliers: 81173


In [69]:
output_path_final = "ncr_ride_bookings_processed.csv"
df_clean.to_csv(output_path_final, index=False)
print(f"Dataset procesado completo guardado en: {output_path_final}")

Dataset procesado completo guardado en: ncr_ride_bookings_processed.csv


# 5. Métricas del negocio

## Métricas con dataset con outliers

In [70]:

print("=== MÉTRICAS DEL NEGOCIO ===\n")

# 1. Total de ingresos
total_revenue = df['booking_value'].sum()
print(f"Total de ingresos: ${total_revenue:,.2f}")

# 2. Distancia promedio
avg_distance = df['ride_distance'].mean()
print(f"Distancia promedio por viaje: {avg_distance:.2f} km")

# 3. Tasa aparente de cancelación
# Contar filas con "cancel" en booking_status
cancelled_bookings = df['booking_status'].str.contains('cancel', case=False, na=False).sum()
total_bookings = len(df)
cancellation_rate = (cancelled_bookings / total_bookings) * 100

print(f"Tasa de cancelación: {cancellation_rate:.2f}%")
print(f"   - Viajes cancelados: {cancelled_bookings:,}")
print(f"   - Total de viajes: {total_bookings:,}")

# Ingreso promedio por viaje
avg_revenue_per_ride = df['booking_value'].mean()
print(f"   - Ingreso promedio por viaje: ${avg_revenue_per_ride:.2f}")

# Distancia total
total_distance = df['ride_distance'].sum()
print(f"   - Distancia total recorrida: {total_distance:,.2f} km")

# Tiempo promedio de viaje
avg_vtat = df['avg_vtat'].mean()
avg_ctat = df['avg_ctat'].mean()
print(f"   - Tiempo promedio VTAT: {avg_vtat:.2f} min")
print(f"   - Tiempo promedio CTAT: {avg_ctat:.2f} min")

# Calificaciones promedio
avg_driver_rating = df['driver_ratings'].mean()
avg_customer_rating = df['customer_rating'].mean()
print(f"   - Calificación promedio conductores: {avg_driver_rating:.2f}/5")
print(f"   - Calificación promedio clientes: {avg_customer_rating:.2f}/5")

# Resumen por estado de reserva
print(f"\nRESUMEN POR ESTADO DE RESERVA:")
status_counts = df['booking_status'].value_counts()
for status, count in status_counts.items():
    percentage = (count / len(df)) * 100
    print(f"   - {status}: {count:,} ({percentage:.1f}%)")

=== MÉTRICAS DEL NEGOCIO ===

Total de ingresos: $51,426,264.00
Distancia promedio por viaje: 24.64 km
Tasa de cancelación: 25.00%
   - Viajes cancelados: 37,191
   - Total de viajes: 148,767
   - Ingreso promedio por viaje: $508.29
   - Distancia total recorrida: 2,493,048.69 km
   - Tiempo promedio VTAT: 8.45 min
   - Tiempo promedio CTAT: 29.15 min
   - Calificación promedio conductores: 4.23/5
   - Calificación promedio clientes: 4.40/5

RESUMEN POR ESTADO DE RESERVA:
   - completed: 92,248 (62.0%)
   - cancelled_by_driver: 26,789 (18.0%)
   - cancelled_by_customer: 10,402 (7.0%)
   - no_driver_found: 10,401 (7.0%)
   - incomplete: 8,927 (6.0%)


## Métricas con dataset sin outliers

In [71]:
print("=== MÉTRICAS DEL NEGOCIO (DATASET LIMPIO) ===\n")
df_analysis = df_clean

print(f"Registros en análisis: {len(df_analysis):,}\n")

# 1. Total de ingresos
total_revenue = df_analysis['booking_value'].sum()
print(f"Total de ingresos: ${total_revenue:,.2f}")

# 2. Distancia promedio
avg_distance = df_analysis['ride_distance'].mean()
print(f"Distancia promedio por viaje: {avg_distance:.2f} km")

# 3. Tasa aparente de cancelación
# Contar filas con "cancel" en booking_status
cancelled_bookings = df_analysis['booking_status'].str.contains('cancel', case=False, na=False).sum()
total_bookings = len(df_analysis)
cancellation_rate = (cancelled_bookings / total_bookings) * 100

print(f"Tasa de cancelación: {cancellation_rate:.2f}%")
print(f"   - Viajes cancelados: {cancelled_bookings:,}")
print(f"   - Total de viajes: {total_bookings:,}")

# Métricas adicionales útiles
print(f"\nMÉTRICAS ADICIONALES:")

# Ingreso promedio por viaje
avg_revenue_per_ride = df_analysis['booking_value'].mean()
print(f"   - Ingreso promedio por viaje: ${avg_revenue_per_ride:.2f}")

# Distancia total
total_distance = df_analysis['ride_distance'].sum()
print(f"   - Distancia total recorrida: {total_distance:,.2f} km")

# Tiempo promedio de viaje
avg_vtat = df_analysis['avg_vtat'].mean()
avg_ctat = df_analysis['avg_ctat'].mean()
print(f"   - Tiempo promedio VTAT: {avg_vtat:.2f} min")
print(f"   - Tiempo promedio CTAT: {avg_ctat:.2f} min")

# Calificaciones promedio
avg_driver_rating = df_analysis['driver_ratings'].mean()
avg_customer_rating = df_analysis['customer_rating'].mean()
print(f"   - Calificación promedio conductores: {avg_driver_rating:.2f}/5")
print(f"   - Calificación promedio clientes: {avg_customer_rating:.2f}/5")

# Resumen por estado de reserva
print(f"\nRESUMEN POR ESTADO DE RESERVA:")
status_counts = df_analysis['booking_status'].value_counts()
for status, count in status_counts.items():
    percentage = (count / len(df_analysis)) * 100
    print(f"   - {status}: {count:,} ({percentage:.1f}%)")

# Comparación con dataset original (si es posible)
if 'df_clean' in locals() and len(df_clean) != len(df):
    print(f"\nCOMPARACIÓN CON DATASET ORIGINAL:")
    print(f"   - Dataset original: {len(df):,} registros")
    print(f"   - Dataset limpio: {len(df_clean):,} registros")
    print(f"   - Diferencia: {len(df) - len(df_clean):,} registros")

=== MÉTRICAS DEL NEGOCIO (DATASET LIMPIO) ===

Registros en análisis: 81,173

Total de ingresos: $37,415,859.00
Distancia promedio por viaje: 25.99 km
Tasa de cancelación: 0.00%
   - Viajes cancelados: 0
   - Total de viajes: 81,173

MÉTRICAS ADICIONALES:
   - Ingreso promedio por viaje: $460.94
   - Distancia total recorrida: 2,109,584.46 km
   - Tiempo promedio VTAT: 8.51 min
   - Tiempo promedio CTAT: 30.04 min
   - Calificación promedio conductores: 4.29/5
   - Calificación promedio clientes: 4.45/5

RESUMEN POR ESTADO DE RESERVA:
   - completed: 81,173 (100.0%)

COMPARACIÓN CON DATASET ORIGINAL:
   - Dataset original: 148,767 registros
   - Dataset limpio: 81,173 registros
   - Diferencia: 67,594 registros
